In [142]:
from minio import Minio

client = Minio('130.245.177.209:9000',               
               access_key='<key>',
               secret_key='<key>',
               secure=False)

#### Add U-Tables

In [ ]:
u100_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u100.csv')
u101_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u101.csv')
u104_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u104.csv')
u200_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u200.csv')
u175_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u175.csv')
u141_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u141.csv')
u108_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u108.csv')
u178_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u178.csv')
u280_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u280.csv')
u210_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u210.csv')
u780_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/u780_NAV-Movement.csv')

#### Add L-Tables

In [ ]:
l220_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l220.csv')
l222_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l222.csv')
l225_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l225.csv')
l235_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l235.csv')
l295_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l295.csv')
l315_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/l315.csv')

#### Add P-Tables

In [ ]:
p200_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p200.csv')
p201_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p201.csv')
p202_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p202.csv')
p204_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p204.csv')
p205_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p205.csv')
p206_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p206.csv')
p207_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p207.csv')
p208_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p208.csv')
p209_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p209.csv')
p210_obj = client.get_object('datalake', 'epapenhausen/Milestone/pdavies/p210.csv')

In [ ]:
import pandas as pd
u100 = pd.read_csv(u100_obj)
u101 = pd.read_csv(u101_obj)
u104 = pd.read_csv(u104_obj)
u200 = pd.read_csv(u200_obj)
u175 = pd.read_csv(u175_obj)
u141 = pd.read_csv(u141_obj)
u108 = pd.read_csv(u108_obj)
u178 = pd.read_csv(u178_obj)
u280 = pd.read_csv(u280_obj)
u210 = pd.read_csv(u210_obj)
u780 = pd.read_csv(u780_obj)

In [ ]:
p200 = pd.read_csv(p200_obj)
p201 = pd.read_csv(p201_obj)
p202 = pd.read_csv(p202_obj)
p204 = pd.read_csv(p204_obj)
p205 = pd.read_csv(p205_obj)
p206 = pd.read_csv(p206_obj)
p207 = pd.read_csv(p207_obj)
p208 = pd.read_csv(p208_obj)
p209 = pd.read_csv(p209_obj)
p210 = pd.read_csv(p210_obj)

In [ ]:
l220 = pd.read_csv(l220_obj)
l222 = pd.read_csv(l222_obj)
l225 = pd.read_csv(l225_obj)
l235 = pd.read_csv(l235_obj)
l295 = pd.read_csv(l295_obj)
l315 = pd.read_csv(l315_obj)

In [ ]:
# Compound unique keys: (Entity Code, Effective Date, Run_ID, Price Run)
def sanitize(table, run_type='Oversight', status=None):
    filt = (table['price_run_type']==run_type)
    if status is not None:
        filt = filt&(table['p_status']==status)
    
    table_san = table[filt]
    drop = [c for c in table_san.columns if table_san[c].nunique() <= 1]+['pcontrol_id']
    return table_san.drop(drop, axis=1)  

In [ ]:
u101_san = sanitize(u101, status='V')
u100_san = sanitize(u100, status='V')

a01 = sanitize(p200)
a02 = sanitize(p201)
a03 = sanitize(p202)

In [ ]:
def nested_summary(raw, keys, data_name):
    ''' Computes summary for categorical attributes using "keys" as a primary key. '''
    categories = [c for c in raw.columns[raw.dtypes==object] if c not in keys]
    cat_data = {str(cat)+'_'+str(level): [] for cat in categories for level in raw[cat].unique()}

    data = {**{k: [] for k in keys}, 'num_'+data_name: [], **cat_data}    
    for k, g in raw.groupby(keys): #'pcontrol_code'):
        n = g.shape[0]
        base_value = g.base_mv_close.sum()
        
        for cd in cat_data:
            data[cd].append(0)
                    
        for cat in categories:        
            #cat_vc = g[cat].value_counts()
            cat_vc = g.groupby(cat)['base_mv_close'].sum()
            for level, value in cat_vc.iteritems():
                index = str(cat)+'_'+str(level)                     
                data[index][-1] = value / base_value
                                
        for i, ky in enumerate(keys):
            data[ky].append(k[i])
        
        data['num_'+data_name].append(n)          
    
    return pd.DataFrame(data)

In [ ]:
keys = ['pcontrol_code', 'effective_date']
u100_data = nested_summary(u100_san, keys, 'assets')

In [ ]:
import pandas as pd
primary_key = ['pcontrol_code', 'effective_date']
product_data = pd.merge(a01, u101_san, how='left', left_on=primary_key, right_on=primary_key)
product_data = pd.merge(product_data, u100_data, how='left', left_on=primary_key, right_on=primary_key)

Specify columns to be dropped as they are not relevant to the analysis

In [ ]:
# drop date columns not relevant to the analysis
datedrop = ['last_updated_date_x', 'last_updated_date_y', 'run_date_and_time', 'value_date', 'effective_date']

# since we will be using the max_tol_lvl to construct the target variable, drop variables that could cause leakage
leakdrop = ['max_tol_lvl', 'holding_max_tol_lvl']

# drop columns indicating the data source
sourcedrop = [c for c in product_data.columns if '_source_' in c]

# drop columns relating to internal pcontrol flags
pcontroldrop = ['integrity_value', 'pcontrol_code'] + [c for c in product_data.columns if 'batch' in c] + [c for c in product_data.columns if c[:2] == 'c_'] + [c for c in product_data.columns if 'updated_by' in c]

drop = datedrop + leakdrop + sourcedrop + pcontroldrop

Extract the Year, Month, and Day from the date columns

In [ ]:
import numpy as np
value_year = product_data['value_date'].apply(lambda x: float(str(x)[:4]) if not np.isnan(x) else x)
value_month = product_data['value_date'].apply(lambda x: float(str(x)[4:6]) if not np.isnan(x) else x)
value_day = product_data['value_date'].apply(lambda x: float(str(x)[6:]) if not np.isnan(x) else x)

eff_year = product_data['effective_date'].apply(lambda x: float(str(x)[:4]) if not np.isnan(x) else x)
eff_month = product_data['effective_date'].apply(lambda x: float(str(x)[4:6]) if not np.isnan(x) else x)
eff_day = product_data['effective_date'].apply(lambda x: float(str(x)[6:]) if not np.isnan(x) else x)

In [ ]:
product_data['value_date_year'] = value_year
product_data['value_date_month'] = value_month
product_data['value_date_day'] = value_day

product_data['effective_date_year'] = eff_year
product_data['effective_date_month'] = eff_month
product_data['effective_date_day'] = eff_day

In [ ]:
product_data['pcontrol_code'].unique()

fund_names = u100_san['pcontrol_code'].unique()
product_data['fund'] = product_data['pcontrol_code'].apply(lambda x: next((f for f in fund_names if f in x), x))

#### A01 Product Validation

In [ ]:
import data_context_map.pattern_miner as pm

# A tolerance breach has occured if max_tol_lvl > 0
data = product_data.copy()
data['breached'] = (data['max_tol_lvl']>0).astype(int)

# drop the occurrences where the previous nav amount is 0
data = data[data['net_nav_amount_prev']!=0]
data.drop(drop, axis=1, inplace=True)

target = 'breached'
out = pm.DataContextMap(data.drop(['fund'], axis=1), target, max_pattern=1000, max_depth=3, mine_type='binary', holdout=0)
out.render()